In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import random
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score, euclidean_distances
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import RandomizedSearchCV
from skopt.space import Real, Integer

import warnings
warnings.filterwarnings('ignore')

# Data Preparation

In [ ]:
df = pd.read_csv('diabetes_klasifikasi.csv')

In [ ]:
df

In [ ]:
df.info()

# Exploratory Data Analysis

In [ ]:
df.describe()

In [ ]:
def plot_null(df):
  plt.figure(figsize=(20,5))
  ax = sns.barplot(x=df.isna().sum().sort_values().index, 
                   y=df.isna().sum().sort_values().values)
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
  ax.bar_label(ax.containers[0])
  plt.show()

In [ ]:
plot_null(df)

In [ ]:
df.duplicated().sum()

In [ ]:
df.iloc[:,:-1].duplicated().sum()

In [ ]:
sns.heatmap(df.corr(), vmin=-1, center=0, vmax=1, annot=True)
plt.show()

## Unique and Count Values

In [ ]:
for i in df.columns:
    print(df.loc[:,i].value_counts())
    print('============================ \n')

In [ ]:
df.columns

## Univariate Analysis

In [ ]:
plt.pie(x=df['Outcome'].value_counts().values, labels=df['Outcome'].value_counts().index, 
        explode=(0,0.1), autopct='%1.2f%%', textprops={'fontsize':15})
plt.title('Ratio Between Two Class In Dependent Variable')
plt.legend(loc='upper right')
plt.show()

In [ ]:
fig, ax = plt.subplots(8,1, figsize=(15,50))
columns = df.iloc[:,:-1].columns

for i, ax in zip(range(len(columns)), ax.flat):
    df[columns[i]].value_counts().plot(kind='bar', ax=ax)
    ax.set_title(columns[i])
    
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(15,10))
columns = df.iloc[:,:-1].columns

for i,ax in zip(range(len(columns)), ax.flat):
    df[columns[i]].value_counts().plot(kind='box', ax=ax)
    
plt.tight_layout()
plt.show()

## Multivariate Analysis

In [ ]:
fig, axs = plt.subplots(8,1, figsize=(10,50))
columns = df.iloc[:,:-1].columns

for i, ax in zip(range(len(columns)), axs.flat):
    sns.boxplot(x='Outcome', y=columns[i], data=df, ax=ax)
    ax.set_title(columns[i])
    
plt.tight_layout()
plt.show()

# Data Cleaning

## Handle Data Outlier

In [ ]:
df.describe()

In [ ]:
columns = df.iloc[:,:-1].columns
for col in columns:
    iqr = df.loc[:, col].quantile(.75) - df.loc[:, col].quantile(.25)
    upper_bound = df.loc[:, col].quantile(.75) + 1.5*iqr

    df.loc[df[col]>upper_bound, col] = upper_bound

In [ ]:
df.describe()

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(15,10))
columns = df.iloc[:,:-1].columns

for i,ax in zip(range(len(columns)), ax.flat):
    df[columns[i]].value_counts().plot(kind='box', ax=ax)
    
plt.tight_layout()
plt.show()

# Data Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop('Outcome', axis=1).values
y = df['Outcome'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

X_train_scaled

# LVQ

## Implementasi algoritma LVQ dalam class

In [ ]:
class LVQ(BaseEstimator, ClassifierMixin):

    def __init__(self, alpha=1e-1, dec_alpha=.5, min_alpha=1e-25, input_size=8, 
                 n_classes=2, epochs=20):
        self.min_alpha = min_alpha
        self.weights_ = np.array([])
        self.input_size_ = input_size
        self.n_classes_ = n_classes
        self.alpha = alpha
        self.dec_alpha = dec_alpha
        self.epochs = epochs
        
        if self.alpha < self.min_alpha:
            raise Exception('alpha harus > min_alpha !')
    
    def get_params(self, deep=True):
        return {'alpha': self.alpha, 
                'dec_alpha': self.dec_alpha,
                'min_alpha': self.min_alpha,
                'input_size': self.input_size_,
                'n_classes': self.n_classes_,
                'epochs': self.epochs}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def winner(self, sample):
        distance0 = euclidean_distances(sample.reshape(1,-1), self.weights_[0:1])
        distance1 = euclidean_distances(sample.reshape(1,-1), self.weights_[1:])

        if distance0[0,0] < distance1[0,0]:
            return 0
        else:
            return 1

    def fit(self, X, y, verbose=1):

        X, y = check_X_y(X, y)

        if self.weights_.size == 0:
            idx = np.random.randint(len(X), size=self.n_classes_)
            self.weights_ = X[idx]
        else:
            None

        for epoch in (tqdm(range(self.epochs)) if verbose == 1 else range(self.epochs)):
            if verbose == 2:
                print('Start Epoch {}'.format(epoch+1))
                print('=============================')
    
            elif verbose == 3:
                print('Start Epoch {}'.format(epoch+1))
                print('Weights on start epoch {0}: {1}'.format(epoch+1, self.weights_))
                print('=============================')
            else:
                None

            for i in range(len(X)):
                pred = self.winner(X[i])
                actual = y[i]
                if actual == pred:
                    for j in range(self.input_size_):
                        self.weights_[pred, j] = self.weights_[pred, j] + self.alpha * (X[i, j] - self.weights_[pred, j])
                else:
                    for j in range(self.input_size_):
                        self.weights_[pred, j] = self.weights_[pred, j] - self.alpha * (X[i, j] - self.weights_[pred, j])

            if self.alpha >= self.min_alpha:
                update_alpha = self.alpha * self.dec_alpha
                if self.min_alpha < update_alpha:
                    self.alpha = update_alpha 
                else:
                    if verbose > 0:
                        print('=======================')
                        print('Alpha minimal tercapai!')
                        print('=======================')
                    
                    return self
                
        return self
    
    def predict(self, X):

        check_is_fitted(self)
        X = check_array(X)

        prediction = np.array([])
        for sample in X:
            prediction = np.append(prediction, self.winner(sample))
        return prediction

## Hyperparameter Tuning LVQ

In [ ]:
params = {'alpha': Real(1e-3, 1),
          'dec_alpha': Real(1e-1, 9e-1),
          'min_alpha': Real(1e-100, 1e-4),
          'epochs': Integer(50, 200)}
rs = RandomizedSearchCV(LVQ(), params, n_iter=200, cv=5, scoring='accuracy', 
                        verbose=0, n_jobs=2)
rs.fit(X_train_scaled, y_train, verbose=0)

In [ ]:
results = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score')
results[['param_alpha', 'param_dec_alpha', 'param_min_alpha', 'param_epochs', 'mean_test_score', 
          'std_test_score', 'rank_test_score']].head(10)

## Training LVQ

In [ ]:
lvq = LVQ(alpha=.07459, dec_alpha=.1, epochs=200, min_alpha=1e-70)
lvq.fit(X_train_scaled, y_train)

## Testing LVQ

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

y_pred = lvq.predict(X_test_scaled)
print(classification_report(y_test, y_pred))
accuracy_score(y_test, y_pred), f1_score(y_test, y_pred)

# LVQ - GA

## Implementasi algoritma LVQ-GA dalam class

In [ ]:
class LVQ_GA(LVQ, BaseEstimator, ClassifierMixin):
    def __init__(self, alpha=1e-1, dec_alpha=.5, min_alpha=1e-25, input_size=8, 
                 n_classes=2, lvq_epochs=20, num_generations=50, sol_per_pop=100,
                 cr=.9, pc=.9, pm=.5):
        self.num_generations = num_generations
        self.sol_per_pop = sol_per_pop
        self.cr = cr
        self.pc = pc
        self.pm = pm
        self.population = np.array([])
        self.fitness = np.array([])
        self.reset_alpha = alpha
        LVQ.__init__(self, alpha, dec_alpha, min_alpha, input_size, n_classes,
                     lvq_epochs)
    
    def get_params(self, deep=False):
        params = LVQ.get_params(self)
        params['num_generations'] = self.num_generations
        params['sol_per_pop'] = self.sol_per_pop
        params['cr'] = self.cr
        params['pc'] = self.pc
        params['pm'] = self.pm
        params['population'] = self.population

        return params
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def create_pop(self, X, y):
        data_0 = X[y == 0]
        data_1 = X[y == 1]
        idx_0 = np.random.randint(0, len(data_0), self.sol_per_pop)
        idx_1 = np.random.randint(0, len(data_1), self.sol_per_pop)
        pop = []
        for i in range(self.sol_per_pop):
            pop.append([data_0[idx_0[i]],data_1[idx_1[i]]])
        
        self.population = np.array(pop)
        
        return self

    def parent_random_selection(self):
        position = np.random.permutation(self.population.shape[0])
        parents = copy.deepcopy(self.population[position[:2]])

        return parents

    def parent_tour_selection(self, K):
        parents = []
        for i in range(2):
            position = np.random.permutation(self.sol_per_pop)[:K]
            max_fitness = self.fitness[position].max()
            parent_idx = np.where(self.fitness == max_fitness)[0][0]
            parent = copy.deepcopy(self.population[parent_idx])
            parents.append(parent)

        return parents
    
    def crossover_2_point(self, p1, p2):
        ps = random.sample(range(1, p1.shape[1] - 1), 2)
        ps.append(0)
        ps.append(p1.shape[1])
        ps = sorted(ps)
        c1, c2 = copy.deepcopy(p1), copy.deepcopy(p2)
        
        for i in range(0, 3):
            if i % 2 == 0:
                continue
            c1[:,ps[i]:ps[i + 1]] = p2[:,ps[i]:ps[i + 1]]
            c2[:,ps[i]:ps[i + 1]] = p1[:,ps[i]:ps[i + 1]]

        return c1, c2
    
    def random_mutation(self, ind):
        mut = copy.deepcopy(ind)

        for i in range(len(mut)):
            mut[i] = mut[i] + random.uniform(-1,1)*(max(mut[i]) - min(mut[i]))
        
        mut = np.array(mut)

        return mut
    
    def run_crossover_mutation(self, parents, generation):

        crossed_offspring = []
        ind1, ind2 = copy.deepcopy(parents[0]), copy.deepcopy(parents[1])
        if random.random() < self.pc:
            # Applying crossover to pair of individuals
            children = self.crossover_2_point(ind1, ind2)
            crossed_offspring.append(children[0])
            crossed_offspring.append(children[1])
        else:
            # Passing individuals further without crossover
            crossed_offspring.append(ind1)
            crossed_offspring.append(ind2)

        crossed_offspring = np.array(crossed_offspring)

        # MUTATION
        mutated = []
        for ind in crossed_offspring:
            if random.random() < self.pm:
                # Applying mutation to an individual
                mutated.append(self.random_mutation(ind))
            else:
                # Passing individual further without mutation
                mutated.append(ind)
            
        mutated = np.array(mutated)
        if generation == 0:
            self.population = np.append(self.population, mutated)\
                                        .reshape(-1, self.n_classes_, 
                                                self.input_size_)
        else:
            self.mutated = np.append(self.mutated, mutated)\
                                     .reshape(-1, self.n_classes_, 
                                              self.input_size_)
            self.population = np.append(self.population, mutated)\
                                        .reshape(-1, self.n_classes_, 
                                                self.input_size_)

        return self
    
    def run_lvq(self, X, y, weights, verbose):
        self.weights_ = copy.deepcopy(weights)
        self.alpha = self.reset_alpha
        LVQ.fit(self, X, y, verbose)
        return self
    
    def eval_fitness(self, y_true, y_pred):
        fitness = accuracy_score(y_true, y_pred)
        return fitness

    def elitism_selection(self, fitness_list, n_child):
        if self.fitness.size == 0:
            self.fitness = copy.deepcopy(fitness_list)
        else:
            self.fitness = np.append(self.fitness, fitness_list)
        idxpop_fitness_zip = list(zip(range(self.sol_per_pop+n_child), self.fitness))
        idxpop_fitness_zip = sorted(idxpop_fitness_zip, key=lambda x: x[1], reverse=True)
        self.idxpop_fitness_zip = np.array(idxpop_fitness_zip)

        idxpop_selected = self.idxpop_fitness_zip[:self.sol_per_pop, 0].astype(int)
        self.fitness = self.idxpop_fitness_zip[:self.sol_per_pop, 1]
        self.population = self.population[idxpop_selected]

        return self        

    def fit(self, X, y, val_data, lvq_verbose=1, ga_verbose=1): # ini belom
        self.create_pop(X, y)

        if val_data != None:
            X_train, X_val, y_train, y_val = X, val_data[0], y, val_data[1]
        else:
            X_train, X_val, y_train, y_val = X.copy(), X.copy(), y.copy(), y.copy()

        n_child = int(self.cr * self.sol_per_pop)
        self.best_fitness_ = 0
        self.best_weights_ = 0

        for generation in range(self.num_generations):    
            
            generation_best_fitness = 0
            
            self.mutated = np.array([])
            for i in range(int(np.floor(n_child/2))):
                if generation == 0:
                    parents = self.parent_random_selection()
                else:
                    parents = self.parent_tour_selection(5)
                self.run_crossover_mutation(parents, generation)

            fitness_list = []

            print('Generation {} Start...'.format(generation+1))
            for ind in tqdm(self.population if generation==0 else self.mutated):
                self.run_lvq(X_train, y_train, ind, lvq_verbose)
                predictions = LVQ.predict(self, X_val)
                fitness = self.eval_fitness(y_val, predictions)

                if fitness > generation_best_fitness:
                    generation_best_fitness = copy.deepcopy(fitness)
                    
                    if fitness > self.best_fitness_:
                        self.best_fitness_ = copy.deepcopy(fitness)
                        self.best_weights_ = copy.deepcopy(self.weights_)

                fitness_list.append(fitness)
            
            print('Best generation fitness:', generation_best_fitness)
            self.elitism_selection(fitness_list, n_child)
        
        print('Best fitness:', self.best_fitness_)
        self.weights_ = copy.deepcopy(self.best_weights_)

        return self
    
    def predict(self, X):
        pred = LVQ.predict(self, X)
        return pred

## Training LVQ-GA 1
Training LVQ-GA dilakukan sebanyak 3 kali, dengan kombinasi hyperparameter yang berbeda-beda

In [ ]:
lvq_ga = LVQ_GA(alpha=.07459, dec_alpha=.1, lvq_epochs=200, 
                min_alpha=1e-70, sol_per_pop=50, 
                num_generations=100, cr=.5, pc=.9, pm=.5)
lvq_ga.fit(X_train_scaled, y_train, 
           val_data=(X_test_scaled, y_test), 
           lvq_verbose=0)

## Testing LVQ-GA 1

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

y_pred = lvq_ga.predict(X_test_scaled)
print(classification_report(y_test, lvq_ga.predict(X_test_scaled)))
accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.74      0.95      0.83        76
           1       0.79      0.38      0.51        40

    accuracy                           0.75       116
   macro avg       0.77      0.66      0.67       116
weighted avg       0.76      0.75      0.72       116



(0.75, 0.5084745762711864, 0.7894736842105263, 0.375)

## Training LVQ-GA 2

In [ ]:
lvq_ga = LVQ_GA(alpha=.07459, dec_alpha=.1, lvq_epochs=200, 
                min_alpha=1e-70, sol_per_pop=30, 
                num_generations=20, cr=.9, pc=.9, pm=.1)
lvq_ga.fit(X_train_scaled, y_train, 
           val_data=(X_test_scaled, y_test), 
           lvq_verbose=0)

Generation 1 Start...


100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [11:01<00:00, 11.81s/it]


Best generation fitness: 0.7597402597402597
Generation 2 Start...


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [05:04<00:00, 11.71s/it]


Best generation fitness: 0.7597402597402597
Generation 3 Start...


...

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [04:36<00:00, 10.65s/it]


Best generation fitness: 0.7597402597402597
Generation 20 Start...


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [04:37<00:00, 10.66s/it]


Best generation fitness: 0.7597402597402597
Best fitness: 0.7597402597402597


LVQ_GA(alpha=7.459000000000029e-70, dec_alpha=0.1, epochs=200, min_alpha=1e-70,
       num_generations=20, pm=0.1,
       population=array([[[-0.83373512, -0.27664283,  0.5765773 ,  1.59524918,
          0.61736523,  0.38299407, -0.84013692, -0.78196499],
        [-1.13880864,  0.75431854, -3.76720748, -1.32277365,
         -0.80806142,  1.63178711,  0.62395952, -0.16392793]],

       [[-0.83373512, -0.27664283,  0.5765773 ,  1.59...
          0.61736523,  0.38299407, -0.84013692, -0.78196499],
        [-1.13880864,  0.75431854, -3.76720748, -1.32277365,
         -0.80806142,  1.63178711,  0.62395952, -0.16392793]],

       [[-0.83373512, -0.27664283,  0.5765773 ,  1.59524918,
          0.61736523,  0.38299407, -0.84013692, -0.78196499],
        [-1.13880864,  0.75431854, -3.76720748, -1.32277365,
         -0.80806142,  1.63178711,  0.62395952, -0.16392793]]]),
       sol_per_pop=30)

## Testing LVQ-GA 2

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.77      0.80        99
           1       0.64      0.75      0.69        55

    accuracy                           0.76       154
   macro avg       0.74      0.76      0.75       154
weighted avg       0.77      0.76      0.76       154



In [ ]:
print(f1_score(y_test,y_pred))

0.6890756302521008


In [ ]:
print(accuracy_score(y_test,y_pred))

0.7597402597402597


In [ ]:
print(precision_score(y_test,y_pred))

0.640625


In [ ]:
print(recall_score(y_test,y_pred))

0.7454545454545455


## Training LVQ-GA 3

In [ ]:
lvq_ga = LVQ_GA(alpha=.07459, dec_alpha=.1, lvq_epochs=200, 
                min_alpha=0.0001, sol_per_pop=100, 
                num_generations=100, cr=.9, pc=.9, pm=.5)
lvq_ga.fit(X_train_scaled, y_train, 
           val_data=(X_test_scaled, y_test), 
           lvq_verbose=0)

Generation 1 Start...


100%|██████████| 190/190 [01:02<00:00,  3.05it/s]


Best generation fitness: 0.8017241379310345
Generation 2 Start...


100%|██████████| 90/90 [00:31<00:00,  2.90it/s]


Best generation fitness: 0.7931034482758621
Generation 3 Start...


...

100%|██████████| 90/90 [00:29<00:00,  3.03it/s]


Best generation fitness: 0.8189655172413793
Generation 100 Start...


100%|██████████| 90/90 [00:29<00:00,  3.03it/s]

Best generation fitness: 0.8189655172413793
Best fitness: 0.8189655172413793


LVQ_GA(alpha=0.0007459000000000001, dec_alpha=0.1, epochs=200, min_alpha=0.0001,
       num_generations=100,
       population=array([[[-3.67209467, -1.58103801, -2.4064254 , ..., -2.81377227,
         -4.07752066, -4.03056896],
        [ 0.59754014,  0.39350935, -5.42492522, ..., -3.25669492,
          0.35485046, -1.02630108]],

       [[-3.67209467, -1.58103801, -2.4064254 , ..., -2.81377227,
         -4.07752066, -4.03056896],
        [ 0....
        [ 1.50183621,  1.29780542, -4.52062915, ..., -2.35239885,
          1.25914653, -0.12200501]],

       [[-0.8723588 ,  1.21869785,  0.39331047, ..., -0.01403641,
         -1.27778479, -1.23083309],
        [ 1.50183621,  1.29780542, -4.52062915, ..., -2.35239885,
          1.25914653, -0.12200501]],

       [[-0.8723588 ,  1.21869785,  0.39331047, ..., -0.01403641,
         -1.27778479, -1.23083309],
        [ 1.50183621,  1.29780542, -4.52062915, ..., -2.35239885,
          1.25914653, -0.12200501]]]))

In [ ]:
lvq_ga.best_weights_

array([[-0.43729137, -0.68401822, -0.11621086, -0.05240362, -0.21205349,
        -0.48836701, -0.26858928, -0.56306414],
       [ 0.95103198,  1.8495751 , -0.91242062, -0.04844099, -0.29778965,
         0.86386935,  0.56021602,  0.1723636 ]])

## Testing LVQ-GA 3

In [ ]:
y_pred = lvq_ga.predict(X_test_scaled)

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.87      0.86        76
           1       0.74      0.72      0.73        40

    accuracy                           0.82       116
   macro avg       0.80      0.80      0.80       116
weighted avg       0.82      0.82      0.82       116



In [ ]:
print(accuracy_score(y_test,y_pred))

0.8189655172413793


In [ ]:
print(f1_score(y_test,y_pred))

0.7341772151898733


In [ ]:
print(recall_score(y_test,y_pred))

0.725
